# MySQL Intro

## Building Blocks

- Database
    - a collection of tables, views, functions, triggers, procedures... are organized for esay access, manipulate, manage

- Schema
  - schema is an abstraction layer above database, in order to provider better control of resources in database, like RBAC, privilege management
  - DCL: `REVOKE; GRANT`
    - `CREATE USER 'newuser'@'localhost' IDENTIFIED BY 'password';`
    - `GRANT ALL PRIVILEGES ON *.* TO 'username'@'localhost' IDENTIFIED BY 'password';`

- Table
  - a row of data sets, properties defined with columns, relationships with other tables defined ny schema.
  - DDL: `CREATE, ALTER, RENAME, DROP, TRUNCATE`
  - DML: `UPDATE, DELETE, INSERT`
  - DQL: `SELECT, SHOW, HELP`
  - TCL/DTL: `START TRANSACTION, SAVEPOINT, COMMIT, ROLLBACK`

- View

- Column/Field
  - a property/attribute to describe datum.
  - typed, constrained, keyed, validated, defaulted...

- Row/Record
  - a collection of selected 
  - a subset of tables
  - business logical, RDB design principle, and performance constrained

- Function

- Stored Procedure

## MySQl Architecture

### Layered Arch

- Architecture
> ![Alt](./MySQL/MySQL-Arch-01.png)            
> ![Alt](./MySQL/MySQL-Arch-02.png)

- Connection Layer
    - Connection Management and Security
- Excution Layer
    - Lock and Concurent
        - lock granularity
        - multiversion concurrency control
    - Cache design
    - Transaction
        - ACID
- Storage Engine Layer
    - Filesystem
    - Varierty of engines for storage
    - logging
    

### Terms

## SQL Langage

### DDL

#### CREATE

create tables
```SQL
CREATE TABLE sakila.film_new LIKE sakila.film;
```


#### ALTER

change storage engine
```SQL
ALTER TABLE mytable ENGINE = InnoDB;
```

```SQL
CREATE TABLE innodb_table LIKE myisam_table;  
ALTER TABLE innodb_table ENGINE=InnoDB;  
INSERT INTO innodb_table SELECT * FROM myisam_table;  
```

```SQL
START TRANSACTION;
INSERT INTO innodb_table SELECT * FROM myisam_table
    WHERE id BETWEEN x AND y;
COMMIT; 
```


change tables
```SQL
ALTER TABLE sakila.film
    MODIFY COLUMN rental_duration TINYINT(3) NOT NULL DEFAULT 5;

ALTER TABLE sakila.film
    ALTER COLUMN rental_duration SET DEFAULT 5;
    
CREATE TABLE sakila.film_new LIKE sakila.film;
ALTER TABLE sakila.film_new
    MODIFY COLUMN rating ENUM('G','PG','PG-13','R','NC-17', 'PG-14')
    DEFAULT 'G';
FLUSH TABLES WITH READ LOCK;
```

- RENAME

- TRUNCATE

- DROP

### DQL

#### SELECT

- SELECT
    - FROM -> WHERE -> GROUP BY -> HAVING -> ORDER BY

#### SHOW

show system infos

```SQL
show global status;
show full processlist;
show innodb status;
SHOW STATUS LIKE 'Last_query_cost';
```

show DBs infos

```SQL
show databases;
show tables;
show tables from SCHEMA|DBNAME;
desc TABLENAME;
show columns from TABLENAME;
show create table TABLENAME;
```

#### HELP

```python
HELP 'command'
```

### DML

#### UPDATE

#### DELETE

#### INSERT

### DTL/TCL

#### START TRANSACTION

#### SAVEPOINT

#### COMMIT

#### ROLLBACK

### DCL

#### REVOKE

#### GRANT

### Built-ins

### Benchmark

- Mearsurements
    - throughput
    - response time/latency
    - concurency
    - scalability
    

# Hands-on

## Setup Envs

### Config 

In [78]:
import mysql.connector
from mysql.connector import Error
from datetime import datetime
import logging
g_dbconfig = {
  'user': 'root',
  'password': 'root1MYSQL',
  'host': '127.0.0.1',
  'database': 'pymysql',
  'raise_on_warnings': True
}
# logging.basicConfig(filename='MySQLDB.log', filemode='w',
#                     level=logging.DEBUG,
#                     format='%(asctime)s-%(levelname)s- %(process)d: %(message)s')

In [79]:
def logger(*arg, **kwargs):
    pass
#     print(*arg, **kwargs)

In [80]:
class MySQLDB(object):
    """
    MySQL Connector/Python Wrapper
    """
    _connection = None
    _instance = None
    _config = None

    @classmethod
    def renew_instance(cls, *args, **kwargs):
        try:
            if cls.connection:
                cls._connection.cursor().close()
                cls._connection.close()
                cls._connection = None
            del cls._instance
            cls._instance = None
        except:
            pass
        cls._instance = object.__new__(cls)
        g_dbconfig.update(kwargs)
        cls._config = g_dbconfig
        return cls._instance
    
    @classmethod
    def __new__(cls, *args, **kwargs):
        if not cls._instance:
            cls._instance = object.__new__(cls)
            logger(f"new obj: {cls._instance}")
        return cls._instance
    
    def __init__(self, *args, **kwargs):
        g_dbconfig.update(kwargs)
        self._config = g_dbconfig
        logger(f"init with: {self._config}")
        
    def renew(self, *args, **kwargs):
        MySQLDB.renew_instance(*args, **kwargs)
        self._config = MySQLDB._config

    def __enter__(self):
        logger(f"__enter__:")
        if self._connection is None:
            try:
                self._connection = mysql.connector.connect(**self._config)

                if self._connection.is_connected():
                    server_info = self._connection.get_server_info()
                    logger(f"Connected to MySQL database, version: {server_info}")

                    cursor = self._connection.cursor()
                    cursor.execute("select database();")
                    db_info = cursor.fetchone()
                    logger(f"You connected to DB: {db_info}  ")

            except Error as err:
                logging.critical("Error while connecting to MySQL: ", err)
                logging.critical("exited......")
        logger(f"enter end...")
        return self._connection

    def __exit__(self, exc_type, exc_val, exc_tb):
        logger(f"__exit__:")
        if self._connection.is_connected():
            self._connection.cursor().execute('commit;')
            self._connection.cursor().close()
            self._connection.close()
            self._connection = None
            logger("MySQL connection is closed")
        logger(f"exit end...")

### Example

In [197]:
with MySQLDB() as db:
    cursor = db.cursor()
    cursor.execute("SHOW TABLES;")
    for table in cursor:
        print(table)

('students',)
('teacherclass',)
('teachers',)
('test',)


## Practice

### csv to mysql with Pandas, MySQL Connector/Python

- setups

In [73]:
from pathlib import Path
import pandas as pd

In [74]:
data_dir = r'C:\Codes\DataScience\Data\mysql'
p = Path(data_dir)
files = [file for file in filter(lambda f: f.is_file(), p.iterdir())]
studentsfile = next(filter(lambda f: 'students' in f.name, files))
studentsfile

WindowsPath('C:/Codes/DataScience/Data/mysql/students.csv')

- Loading Data

In [75]:
data = pd.read_csv(str(studentsfile))

- Clean Data

In [76]:
datefmt = pd.DataFrame([d.split('/') for d in data['entrydate']])
datefmt = datefmt.iloc[:, [2, 0, 1]]
dates = [ '-'.join(tuple(d)) for d in datefmt.loc[:].values ]
data['entrydate'] = dates
data

,studentid,stname,email,birth,entrydate,gender,classnum
0,1,Glen,glent@gmail.com,19901201,2002-4-9,M,1
1,2,Yinging,yirmic302@gmail.com,19901201,2003-6-11,F,2
2,3,Lendor,kude1035@gmail.com,19910312,2000-9-5,M,2
3,4,Tomocro,troman.telcon@gmail.com,19900921,2002-8-20,M,2
4,5,Keren,tl.nesleeza@hotmail.com,19891102,2003-9-2,M,2
5,6,Mafammia,mafuupink@yahoo.ca,19910507,2001-11-27,F,1
6,7,Lina,Lina.centen1023@gmail.com,19911209,2003-3-3,M,3
7,8,Cather,clarkying@yahoo.com,19921222,2005-11-1,M,1
8,9,Minko,mk02@yahoo.com,19901201,2001-11-11,M,2
9,10,Glen,glent@gmail.com,19901023,2002-4-9,M,1


- write to MySQL

In [77]:
with MySQLDB() as db:
    cursor = db.cursor()
    stmt = f'insert into students values (%s, %s, %s, %s, %s, %s, %s)'
    for row in data.loc[:].values:
        record = tuple(row)        
        cursor.execute(stmt, record)
        print('inserting: ', record)

NameError: name 'MySQLDB' is not defined

- check writeback

In [143]:
sqlstmt = '''
select * from students;
'''
with MySQLDB() as db:
    cursor = db.cursor()
    cursor.execute(sqlstmt)
    for row in cursor:
        print(row)

(1, 'Glen', 'glent@gmail.com', datetime.date(1990, 12, 1), datetime.date(2002, 4, 9), 'M', 1)
(2, 'Yinging', 'yirmic302@gmail.com', datetime.date(1990, 12, 1), datetime.date(2003, 6, 11), 'F', 2)
(3, 'Lendor', 'kude1035@gmail.com', datetime.date(1991, 3, 12), datetime.date(2000, 9, 5), 'M', 2)
(4, 'Tomocro', 'troman.telcon@gmail.com', datetime.date(1990, 9, 21), datetime.date(2002, 8, 20), 'M', 2)
(5, 'Keren', 'tl.nesleeza@hotmail.com', datetime.date(1989, 11, 2), datetime.date(2003, 9, 2), 'M', 2)
(6, 'Mafammia', 'mafuupink@yahoo.ca', datetime.date(1991, 5, 7), datetime.date(2001, 11, 27), 'F', 1)
(7, 'Lina', 'Lina.centen1023@gmail.com', datetime.date(1991, 12, 9), datetime.date(2003, 3, 3), 'M', 3)
(8, 'Cather', 'clarkying@yahoo.com', datetime.date(1992, 12, 22), datetime.date(2005, 11, 1), 'M', 1)
(9, 'Minko', 'mk02@yahoo.com', datetime.date(1990, 12, 1), datetime.date(2001, 11, 11), 'M', 2)
(10, 'Glen', 'glent@gmail.com', datetime.date(1990, 10, 23), datetime.date(2002, 4, 9), 'M',

### Pandas r/w to mysql with SQLAlchemy

- setup

In [26]:
from pathlib import Path
import pandas as pd
from pandas.io import sql
from sqlalchemy import create_engine

In [27]:
alchemyconfig = {
    'user': 'root',
    'passw': 'root1MYSQL',
    'host': '127.0.0.1',
    'port': 3306,
    'database': 'pandasdb'
}

alchemyURL= 'mysql+mysqlconnector://{user}:{passw}@{host}/{database}'.format(**alchemyconfig)

mydb = create_engine(alchemyURL, echo=False)

In [3]:
data_dir = r'C:\Codes\DataScience\Data\mysql'
p = Path(data_dir)
files = [file for file in filter(lambda f: f.is_file(), p.iterdir())]
studentsfile = next(filter(lambda f: 'students' in f.name, files))
studentsfile

WindowsPath('C:/Codes/DataScience/Data/mysql/students.csv')

- Loading to DataFrame

In [214]:
mydata = pd.read_csv(str(studentsfile))
mydata

,studentid,stname,email,birth,entrydate,gender,classnum
0,1,Glen,glent@gmail.com,19901201,4/9/2002,M,1
1,2,Yinging,yirmic302@gmail.com,19901201,6/11/2003,F,2
2,3,Lendor,kude1035@gmail.com,19910312,9/5/2000,M,2
3,4,Tomocro,troman.telcon@gmail.com,19900921,8/20/2002,M,2
4,5,Keren,tl.nesleeza@hotmail.com,19891102,9/2/2003,M,2
5,6,Mafammia,mafuupink@yahoo.ca,19910507,11/27/2001,F,1
6,7,Lina,Lina.centen1023@gmail.com,19911209,3/3/2003,M,3
7,8,Cather,clarkying@yahoo.com,19921222,11/1/2005,M,1
8,9,Minko,mk02@yahoo.com,19901201,11/11/2001,M,2
9,10,Glen,glent@gmail.com,19901023,4/9/2002,M,1


- write to MySQL

In [216]:
mydata.to_sql(name='students', con=mydb, if_exists = 'fail', index=False)

In [220]:
pd.read_sql_query("desc students", mydb)

,Field,Type,Null,Key,Default,Extra
0,studentid,bigint(20),YES,,None,
1,stname,text,YES,,None,
2,email,text,YES,,None,
3,birth,bigint(20),YES,,None,
4,entrydate,text,YES,,None,
5,gender,text,YES,,None,
6,classnum,bigint(20),YES,,None,


In [219]:
pd.read_sql_query("SELECT * FROM students", mydb)

,studentid,stname,email,birth,entrydate,gender,classnum
0,1,Glen,glent@gmail.com,19901201,4/9/2002,M,1
1,2,Yinging,yirmic302@gmail.com,19901201,6/11/2003,F,2
2,3,Lendor,kude1035@gmail.com,19910312,9/5/2000,M,2
3,4,Tomocro,troman.telcon@gmail.com,19900921,8/20/2002,M,2
4,5,Keren,tl.nesleeza@hotmail.com,19891102,9/2/2003,M,2
5,6,Mafammia,mafuupink@yahoo.ca,19910507,11/27/2001,F,1
6,7,Lina,Lina.centen1023@gmail.com,19911209,3/3/2003,M,3
7,8,Cather,clarkying@yahoo.com,19921222,11/1/2005,M,1
8,9,Minko,mk02@yahoo.com,19901201,11/11/2001,M,2
9,10,Glen,glent@gmail.com,19901023,4/9/2002,M,1


### Batch load csv to mysql with pandas

- setup

In [29]:
from pathlib import Path
import pandas as pd
from pandas.io import sql
from sqlalchemy import create_engine

In [30]:
alchemyconfig = {
    'user': 'root',
    'passw': 'root1MYSQL',
    'host': '127.0.0.1',
    'port': 3306,
    'database': 'pandasdb'
}

alchemyURL= 'mysql+mysqlconnector://{user}:{passw}@{host}/{database}'.format(**alchemyconfig)

mydb = create_engine(alchemyURL, echo=False)
readengine = create_engine(alchemyURL, echo=False)

In [30]:
data_dir = r'C:\Codes\DataScience\Data\mysql'
p = Path(data_dir)
files = [file for file in filter(lambda f: f.is_file(), p.iterdir())]

In [59]:
for file in files:
    tmp_df = pd.read_csv(str(file))
    try:
        tmp_df.to_sql(name=file.stem, con=mydb, if_exists = 'fail', index=False)
        print(f'write {file.stem} to mysql!')
    except:
        df = pd.read_sql_query(f"show tables like '{file.stem}'", readengine)
        if df.empty:
            print(f'failed to write "{file.stem}" to mysql!')
        else:
            print(f'{file.stem} exists, pass loading.')
        continue

dealer exists, pass loading.
goods exists, pass loading.
goods_customer exists, pass loading.
goods_promotion exists, pass loading.
goods_sell exists, pass loading.
retailcusids exists, pass loading.
retailtransactions exists, pass loading.
retail_cus_ids exists, pass loading.
retail_transactions exists, pass loading.
sales_employees exists, pass loading.
sale_records exists, pass loading.
students exists, pass loading.
subjects exists, pass loading.
subjects_add exists, pass loading.
subscores exists, pass loading.
teacherclass exists, pass loading.
teachers exists, pass loading.
teachers_add exists, pass loading.


In [60]:
df = pd.read_sql_query("show tables", readengine)
df

,Tables_in_pandasdb
0,dealer
1,goods
2,goods_customer
3,goods_promotion
4,goods_sell
5,retail_cus_ids
6,retail_transactions
7,retailcusids
8,retailtransactions
9,sale_records


In [61]:
pd.read_sql_query("desc retailcusids", readengine)

,Field,Type,Null,Key,Default,Extra
0,customerid,bigint(20),YES,,None,
1,age,bigint(20),YES,,None,
2,gender,text,YES,,None,


### SQL Practice

In [72]:
pd.read_sql_query("select * from retailcusids where gender in ('F','M') and age<25 limit 10;", readengine)

,customerid,age,gender
0,27,22,M
1,695,23,F
2,746,24,F
3,752,17,M
4,770,24,M
5,937,24,F
6,997,23,M
7,1011,21,F
8,1053,22,F
9,1081,23,F


In [69]:
pd.read_sql_query("select * from retailcusids where gender='F' and age between 35 and 40 limit 10;", readengine)

,customerid,age,gender
0,1,37,F
1,2,40,F
2,8,39,F
3,9,39,F
4,12,35,F
5,13,38,F
6,16,39,F
7,21,36,F
8,23,38,F
9,26,40,F


In [66]:
pd.read_sql_query("select * from retailcusids where gender='M' and age>=40 limit 10", readengine)

,customerid,age,gender
0,7,44,M
1,17,43,M
2,19,41,M
3,30,42,M
4,44,47,M
5,52,40,M
6,53,40,M
7,56,44,M
8,62,44,M
9,66,47,M


In [31]:
pd.read_sql_query("show global variables", readengine)

,Variable_name,Value
0,activate_all_roles_on_login,OFF
1,auto_generate_certs,ON
2,auto_increment_increment,1
3,auto_increment_offset,1
4,autocommit,ON
...,...,...
525,version_compile_machine,x86_64
526,version_compile_os,Win64
527,version_compile_zlib,1.2.11
528,wait_timeout,28800


In [32]:
pd.read_sql_query("show global status", readengine)

,Variable_name,Value
0,Aborted_clients,0
1,Aborted_connects,0
2,Acl_cache_items_count,0
3,Binlog_cache_disk_use,0
4,Binlog_cache_use,0
...,...,...
423,Threads_connected,1
424,Threads_created,1
425,Threads_running,2
426,Uptime,45567


In [36]:
pd.read_sql_query("show full processlist", readengine)

,Id,User,Host,db,Command,Time,State,Info
0,4,event_scheduler,localhost,None,Daemon,46366,Waiting on empty queue,None
1,8,root,localhost:8585,pandasdb,Query,0,starting,show full processlist
2,11,root,localhost:8614,None,Sleep,22,,None
3,12,root,localhost:8615,None,Sleep,22,,None


In [39]:
pd.read_sql_query("", readengine)

Exception during reset or similar
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sqlalchemy\pool\base.py", line 680, in _finalize_fairy
    fairy._reset(pool)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sqlalchemy\pool\base.py", line 867, in _reset
    pool._dialect.do_rollback(self)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sqlalchemy\dialects\mysql\base.py", line 2253, in do_rollback
    dbapi_connection.rollback()
  File "C:\ProgramData\Anaconda3\lib\site-packages\mysql\connector\connection_cext.py", line 335, in rollback
    self._cmysql.rollback()
_mysql_connector.MySQLInterfaceError: Commands out of sync; you can't run this command now


ResourceClosedError: This result object does not return rows. It has been closed automatically.

# END